In [ ]:
!pip install transformers
!pip install datasets


In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset

def process_csv(file_path):
    df = pd.read_csv("/content/questionsv4 (1).csv")
    qa_pairs = []

    for index, row in df.iterrows():
        question = row['question']
        answer = row['answer']
        qa_pairs.append(f"Question: {question}\nAnswer: {answer}\n")

    return qa_pairs

def load_dataset(file_path, tokenizer):
    qa_pairs = process_csv(file_path)
    tokenized_dataset = tokenizer(qa_pairs, truncation=True,
                                  padding='max_length', max_length=128,
                                  return_tensors="pt")
    dataset = Dataset.from_dict(tokenized_dataset)
    return dataset

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)



In [ ]:
# Save the fine-tuned model
model.save_pretrained("fine_QA_gpt2")

In [ ]:
!pip install pyTelegramBotAPI

In [ ]:
import telebot
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained("fine_QA_gpt2")

# Instantiate the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Replace "YOUR_API_TOKEN" with your Telegram Bot API token
API_TOKEN = "6017018320:AAG-GSgHSU3aOsPdAfOVnsq3BoItSVXto6g"
bot = telebot.TeleBot(API_TOKEN)

@bot.message_handler(commands=['start'])
def start(message):
    welcome_message = "Hello,Welcome to virtual assistant Farmer Bot.I can answer questions about our chatbot platform.What can I help you with? "
    bot.send_message(message.chat.id, welcome_message)

@bot.message_handler(func=lambda message: True)
def answer_question(message):
    question = message.text

    # Generate the answer using the fine-tuned GPT-2 model
    answer = ask_question(question, fine_tuned_model, tokenizer)

    # Send the answer back to the user
    bot.send_message(message.chat.id, answer)

def ask_question(question, model, tokenizer, max_length=128, num_return_sequences=1):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True)
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=3,
        do_sample=True,
        temperature=1.0,
        top_k=50,
        top_p=0.9,
        early_stopping=True,
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.replace(prompt, "").strip()

    # Truncate the answer after the first newline character
    answer = answer.split("\n")[0]

    return answer

# Start the bot
bot.polling()




2023-09-28 07:22:11,268 (__init__.py:1083 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
2023-09-28 07:22:11,273 (__init__.py:1085 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1073, in __threaded_polling
    polling_thread.raise_exceptions()
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 108, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 90, in run
    task(*args, **kwargs)
  